In [3]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA

In [5]:
spy_data = pd.read_csv("../data/spy_daily01.csv")
spy_data.head()

,to_datetime,open,high,low,close,volume
0,2018-10-15 01:00:00.000000,275.55,277.04,274.30,274.40,102263717
1,2018-10-16 01:00:00.000000,276.60,280.82,276.07,280.40,118255834
2,2018-10-17 01:00:00.000000,280.44,281.15,277.56,280.45,110625987
3,2018-10-18 01:00:00.000000,279.40,280.07,274.97,276.40,134557530
4,2018-10-19 01:00:00.000000,277.13,279.30,275.47,276.25,139901630


In [6]:
model_data = spy_data["close"]

In [9]:
mod_arma = ARIMA(model_data, order=(1,1,1))
res_arma = mod_arma.fit()

# Print out summary information on the fit
print(res_arma.summary())

                               SARIMAX Results                                
Dep. Variable:                  close   No. Observations:                  543
Model:                 ARIMA(1, 1, 1)   Log Likelihood               -1568.544
Date:                Mon, 14 Dec 2020   AIC                           3143.087
Time:                        17:02:11   BIC                           3155.973
Sample:                             0   HQIC                          3148.126
                                - 543                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.5102      0.062     -8.178      0.000      -0.633      -0.388
ma.L1          0.2888      0.070      4.121      0.000       0.151       0.426
sigma2        19.1055      0.514     37.138      0.0